In [ ]:
import numpy as np
from sklearn import datasets

def culc_prob(features, param_matrix):  #クラスは3つのみであるとする
    s_matrix = np.dot(param_matrix, features.T)
    s_matrix = s_matrix.T #種類ごとのソフトマックススコアを150インスタンス分格納したnp.array
    prob_matrix = np.zeros((len(s_matrix), 3))
    for i in range(len(s_matrix)):
        prob_matrix[i, 0] = np.exp(s_matrix[i, 0]) / (np.exp(s_matrix[i, 0]) + np.exp(s_matrix[i, 1]) + np.exp(s_matrix[i, 2]))
        prob_matrix[i, 1] = np.exp(s_matrix[i, 1]) / (np.exp(s_matrix[i, 0]) + np.exp(s_matrix[i, 1]) + np.exp(s_matrix[i, 2]))
        prob_matrix[i, 2] = np.exp(s_matrix[i, 2]) / (np.exp(s_matrix[i, 0]) + np.exp(s_matrix[i, 1]) + np.exp(s_matrix[i, 2]))
    return prob_matrix

def culc_CELF(instance_class, prob_matrix):
    J = 0
    for i in range(len(instance_class)):
        J -= np.log(prob_matrix[i, instance_class[i]])
    J /= len(instance_class)
    return J

def culc_gradient_descent_vector(features, instance_class, prob_matrix):  #クラス,パラメータ数がともに3つのみであるとする
    vector_matrix = np.zeros((3, 3))
    y_matrix = np.zeros((len(features), 3))
    for i in range(len(features)):
        y_matrix[i, instance_class[i]] = 1
    subst_matrix = prob_matrix - y_matrix
    for i in range(len(features)):
        vector_matrix[0] += subst_matrix[i, 0] * features[i]
        vector_matrix[1] += subst_matrix[i, 1] * features[i]
        vector_matrix[2] += subst_matrix[i, 2] * features[i]
    vector_matrix /= len(features)
    return vector_matrix

iris = datasets.load_iris()
X = iris['data'][:, 2:] #150インスタンス分の花弁の長さと花弁の幅を格納したnp.array
y = iris['target'] #150インスタンス分の種類を示す整数を格納したnp.array。0:セトサ、1:バージニカ、2:バーシクル
X = np.insert(X, 0, 1, axis=1) #バイアス項を考えて、先頭に1のみの列を追加
np.random.seed(1888)
train_index = np.sort(np.random.choice(np.arange(len(X)), size = round(0.8 * len(X)), replace = False))
valid_index = np.sort(np.random.choice(np.setdiff1d(np.arange(len(X)), train_index), size = round(0.1 * len(X)), replace = False))
test_index = np.setdiff1d(np.setdiff1d(np.arange(len(X)), train_index), valid_index)
X_train_list = []
y_train_list = []
X_valid_list = []
y_valid_list = []
X_test_list = []
y_test_list = []
for num in train_index:
    X_train_list.append(X[num])
    y_train_list.append(y[num])
for num in valid_index:
    X_valid_list.append(X[num])
    y_valid_list.append(y[num])
for num in test_index:
    X_test_list.append(X[num])
    y_test_list.append(y[num])
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)
X_valid = np.array(X_valid_list)
y_valid = np.array(y_valid_list)
X_test = np.array(X_test_list)
y_test = np.array(y_test_list)
print(X_train)
print(y_train)
print(X_valid)
print(y_valid)
parameter_matrix = np.zeros((3, 3))
parameter_list = []
parameter_list.append(parameter_matrix)
CELF_train_list = []
CELF_valid_list = []
epoch = 0
count = 0 #損失関数が良化しない連続回数
learning_rate = 0.2
output_parameter_matrix = np.zeros((3, 3))

while(True):
    probability_train_matrix = culc_prob(X_train, parameter_matrix)
    probability_valid_matrix = culc_prob(X_valid, parameter_matrix)
    CELF_train = culc_CELF(y_train, probability_train_matrix)
    CELF_valid = culc_CELF(y_valid, probability_valid_matrix)
    CELF_train_list.append(CELF_train)
    CELF_valid_list.append(CELF_valid)
    print(CELF_train, CELF_valid)
    gd_vector = culc_gradient_descent_vector(X_train, y_train, probability_train_matrix)
    epoch += 1
    parameter_matrix -= learning_rate * gd_vector
    parameter_list.append(parameter_matrix)
    if epoch == 1:
        pass
    elif CELF_valid_list[epoch - 1] > CELF_valid_list[epoch - 2]:
        count += 1
        if count == 5:
            print("最良のパラメータに達しました")
            print(f"エポック数:{epoch -5 }")
            print("パラメータは以下")
            print(parameter_list[epoch - 5])
            output_parameter_matrix = parameter_list[epoch - 5]
            break
        else:
            continue
    elif epoch == 10 ** 5:
        print("既定のエポック数に達したので、探索を終了しました")
        print("パラメータは以下")
        print(parameter_list[epoch])
        output_parameter_matrix = parameter_list[epoch]
        break
    else:
        pass

probability_test_matrix = culc_prob(X_test, output_parameter_matrix)

correct = 0
error = 0
for i in range(len(X_test)):
    estimated_class = np.argmax(probability_test_matrix[i])
    if estimated_class == y_test[i]:
        correct += 1
    else:
        error += 1
print(f"予測精度: {correct / (correct + error)}")